## Keras model - Deeper2
---- 
* **Why Keras ? **


 > 실질적으로 풀려는 문제가 풀리는 문제 일까 아닐까를 정의가 안된 시점에서, 얕은 모델 부터 깊고 복잡한 모델까지 모두 실험해 보기 위해서 모델을 만들고 러닝 하여, 결과를 확인하는 과정에 사이클을 짧게 하기위함이 첫번째 이유이고, 두번째는 데이터를 직접 resizing하지 않아서 tfrecord로 파싱하는데 오류가 있었기 때문이다. 
 
* **현재 모델**
> **VGG alike** : 맨 처음 1x1conv layer로 gray scals 을 만들고 다음 VGG와 비슷한 구조로 conv을 수행한 후 FCN으로 넘어간다. 


> Deeper와 다른점 
 1.Dense layer 한 개 추가 
 2.1x1 제거 


In [41]:
import numpy as np 
import pickle 

# load dataset 
with open('../sample_image/final_df.bin', 'rb') as f :
    crop_img = pickle.load(f)

# X = np.array([i for i in crop_img.crop_img_05])
# y = crop_img.weight.values

X = np.array([i for i in crop_img.crop_images])
y = crop_img.weight.values

In [28]:
crop_img.columns
crop_img.crop_img_05[0].shape

(125, 275, 3)

In [42]:
#Parameters 

BATCH_SIZE = 24

Optimizers = ["adagrad", "adadelta", "adam", "adamax", ""]

In [43]:
### test/train split 
idx = list(np.random.randint(0, len(X), int(len(X) * 0.8)))
test_idx = list(set(list(range(len(X)))) - set(idx))

X_train = X[idx]
y_train = y[idx]
X_test = X[test_idx]
y_test = y[test_idx]

In [44]:
# image augmentation 
from keras.preprocessing.image import ImageDataGenerator

gen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

gen.fit(X_train, augment=True)

In [45]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from keras.layers import Dropout
from keras import regularizers
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.regularizers import l2


def cnn_model():
    model = Sequential()
    #  first layer for making images to the gray scale images, more precisely 
    model.add(Conv2D(1, (1, 1), activation="relu", input_shape=(250, 550, 3), padding='same', kernel_regularizer=l2(0.001)))

    # 
#     model.add(Conv2D(32, (7, 7), activation="relu", padding='valid', input_shape=(250, 550, 3), kernel_regularizer=l2(0.001)))
#     model.add(Conv2D(32, (7, 7), activation="relu", padding='valid'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling2D())

    model.add(Conv2D(32, (7, 7), activation="relu", padding='valid', kernel_regularizer=l2(0.001)))
    model.add(Conv2D(32, (7, 7), activation="relu", padding='valid', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D())
    
    model.add(Conv2D(32, (7, 7), activation="relu", padding='valid', kernel_regularizer=l2(0.001)))
    model.add(Conv2D(32, (7, 7), activation="relu", padding='valid', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D())

    model.add(Conv2D(64, (3, 3), activation="relu", padding='valid', kernel_regularizer=l2(0.001)))
    model.add(Conv2D(64, (3, 3), activation="relu", padding='valid', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D())
    
    model.add(Conv2D(64, (3, 3), activation="relu", padding='valid', kernel_regularizer=l2(0.001)))
    model.add(Conv2D(64, (3, 3), activation="relu", padding='valid', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D())

    model.add(Flatten())
    model.add(Dense(500, activation="relu", kernel_regularizer=l2(0.001)))
    model.add(Dropout(0.5))
#     model.add(Dense(200, activation="relu", kernel_regularizer=l2(0.001)))
#     model.add(Dropout(0.5))
    model.add(Dense(1, activation="relu"))
    return model 



In [46]:
from keras.utils.training_utils import multi_gpu_model

model = cnn_model()
model = multi_gpu_model(model, gpus=2)
model.compile(loss='mean_squared_error', optimizer='adadelta', metrics=['mae'])

In [47]:
%%time
hist = model.fit_generator(gen.flow(X_train, y_train, batch_size=BATCH_SIZE), 
                           steps_per_epoch=len(X_train)/BATCH_SIZE,
                          epochs=100, validation_data=(X_test, y_test), verbose=2)

Epoch 1/100
 - 27s - loss: 2920.1767 - mean_absolute_error: 42.9207 - val_loss: 68929266249.1429 - val_mean_absolute_error: 261807.8460
Epoch 2/100
 - 19s - loss: 753.9675 - mean_absolute_error: 21.2948 - val_loss: 22667057834.6667 - val_mean_absolute_error: 150107.8839
Epoch 3/100
 - 19s - loss: 506.3792 - mean_absolute_error: 17.8896 - val_loss: 22811450416.7619 - val_mean_absolute_error: 150575.1540
Epoch 4/100
 - 19s - loss: 421.6859 - mean_absolute_error: 16.3719 - val_loss: 9550982826.6667 - val_mean_absolute_error: 97398.0130
Epoch 5/100
 - 19s - loss: 438.0677 - mean_absolute_error: 16.6106 - val_loss: 22672203678.4762 - val_mean_absolute_error: 150055.1815
Epoch 6/100
 - 19s - loss: 406.9072 - mean_absolute_error: 15.8965 - val_loss: 24626335353.9048 - val_mean_absolute_error: 156369.3728
Epoch 7/100
 - 18s - loss: 538.5064 - mean_absolute_error: 18.5986 - val_loss: 73210954703.2381 - val_mean_absolute_error: 269742.3006
Epoch 8/100
 - 18s - loss: 457.2095 - mean_absolute_erro

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline


plt.figure(figsize=(9,9))
plt.plot(hist.history["loss"])
plt.plot(hist.history["val_mean_absolute_error"])
plt.show()

In [ ]:
r = 2
print("prediction : {}, Truth : {} ".format(model.predict(X_test)[r], y_test[r]))